In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import json
import os
import time


In [2]:
# Configurando nav
options = Options()
options.add_argument("--disable-gpu")
options.page_load_strategy = 'eager'  # Não espera o carregamento completo
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(12)
driver.maximize_window()

In [23]:
# Arquivo JSON
json_file = 'noticias.json'

# Verifica se o arquivo já existe e carrega o conteúdo, caso contrário cria uma lista vazia
if os.path.exists(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        noticias = json.load(file)
else:
    noticias = []

In [25]:
link = r"https://somosfanaticos.fans/br/noticias-recentes"
driver.get(link)

In [4]:
links = []
for i in range(10):
    link = driver.find_element('xpath', f'//*[@id="__next"]/main/div[2]/div/div/article[{i+1}]/figure/a').get_attribute('href')
    links.append(link)

KeyboardInterrupt: 

In [ ]:
# Define o número máximo de tentativas
# Somos fanáticos
max_tentativas = 2

# Número de tentativas já realizadas
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Carrega o arquivo JSON e os títulos existentes a cada iteração para garantir que tudo está sendo atualizado
        if os.path.exists(json_file):
            with open(json_file, 'r', encoding='utf-8') as file:
                noticias = json.load(file)
        
        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}
        driver.get("https://somosfanaticos.fans/br/noticias-recentes")
        link = driver.find_element('xpath', f'//*[@id="__next"]/main/div[2]/div/div/article[1]/figure/a').get_attribute('href')
        
        ap = driver.find_element('xpath', '/html/body/div[2]/main/div[2]/div/div/article[1]/div/a/span').get_attribute('innerText')
        ap = ap.upper()
        x = 2
        while ap == "APOSTAS":
            link = driver.find_element('xpath', f'//*[@id="__next"]/main/div[2]/div/div/article[{x}]/figure/a').get_attribute('href')
            ap = driver.find_element('xpath', f'/html/body/div[2]/main/div[2]/div/div/article[{x}]/div/a/span').get_attribute('innerText')
            ap = ap.upper()
            x += 1
        driver.get(link)
        titulo = driver.find_element('xpath', '/html/body/main/div[1]/section/article/div[1]/div[2]/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', f'/html/body/main/div[1]/section/article/div[1]/div[2]/h2').get_attribute('innerText')
            autor = driver.find_element('xpath', f'/html/body/main/div[1]/section/article/div[1]/div[2]/div/div[1]/div').get_attribute('innerText')
            try:
                img = driver.find_element("xpath", f'/html/body/main/div[1]/section/article/div[1]/figure/img').get_attribute('srcset')
                descrimg = driver.find_element("xpath", f'/html/body/main/div[1]/section/article/div[1]/figure/img').get_attribute('alt')
            except:
                img = "Sem imagem"
                descrimg = "Sem imagem"
            conteudo = driver.find_elements('xpath', f'/html/body/main/div[1]/section/article//p | /html/body/main/div[1]/section/article//h2 | /html/body/main/div[1]/section/article//h3')

            conteudo_filtrado = []  # Lista para armazenar o conteúdo filtrado
            skip_next = False  # Flag para pular o próximo índice

            for Y, x in enumerate(conteudo):    
                text = x.text.strip()
            
                if skip_next:  # Se a flag estiver ativada, pula o índice
                    skip_next = False
                    continue

                # Ignorar conteúdo com base nas condições
                if text == "":  # Verifica se o conteúdo é vazio
                    print(f"Ignorando índice {Y} devido a conteúdo vazio")
                    continue

                if "JÁ VOTARAM" in text:
                    print(f"Ignorando índice {Y} devido a 'JÁ VOTARAM'")
                    continue 

                if "VEJA TAMBÉM" in text:
                    print(f"Ignorando índice {Y} e o próximo devido a 'VEJA TAMBÉM'")
                    skip_next = True  # Define a flag para pular o próximo índice
                    continue

                if "LEIA TAMBÉM" in text:
                    print(f"Ignorando índice {Y} e o próximo devido a 'VEJA TAMBÉM'")
                    skip_next = True  # Define a flag para pular o próximo índice
                    break
                
                if "dizem os Fanáticos" in text:
                    print(f"Ignorando índice {Y} devido a 'dizem os Fanáticos'")
                    continue

                if "Siga o canal do Somos" in text:
                    print(f"Ignorando índice {Y} devido a 'Siga o canal do Somos'")
                    continue
            
                conteudo_filtrado.append(text)  # Adiciona o conteúdo

            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_filtrado
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Buscando no próximo site...")


In [ ]:
# Define o número máximo de tentativas
# Antenados no futebol
max_tentativas = 2

# Número de tentativas já realizadas
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        driver.get("https://www.antenadosnofutebol.com.br/latest-news")
        link = driver.find_element('xpath', f'//*[@id="__next"]/main/div[2]/div/div/article[1]/figure/a').get_attribute('href')
        ap = driver.find_element('xpath', '//*[@id="__next"]/main/div[2]/div/div/article[1]/div/a/span').get_attribute('innerText')
        ap = ap.upper()
        x = 2
        while ap == "APOSTAS":
            link = driver.find_element('xpath', f'//*[@id="__next"]/main/div[2]/div/div/article[{x}]/figure/a').get_attribute('href')
            ap = driver.find_element('xpath', f'//*[@id="__next"]/main/div[2]/div/div/article[{x}]/div/a/span').get_attribute('innerText')
            ap = ap.upper()
            x += 1

        driver.get(link)
        titulo = driver.find_element('xpath', f'/html/body/main/div[2]/section/article/div[1]/div[2]/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', f'/html/body/main/div[2]/section/article/div[1]/div[2]/h2').get_attribute('innerText')
            autor = driver.find_element('xpath', f'/html/body/main/div[2]/section/article/div[1]/div[2]/div/div[1]/div/p[1]/a').get_attribute('innerText')
            try:
                img = driver.find_element("xpath", f'/html/body/main/div[2]/section/article/div[1]/figure/img').get_attribute('srcset')
                descrimg = driver.find_element("xpath", f'/html/body/main/div[2]/section/article/div[1]/figure/figcaption').get_attribute('innerText')
            except:
                img = "Sem imagem"
                descrimg = "Sem imagem"
            conteudo = driver.find_elements('xpath', f'/html/body/main/div[2]/section/article//p | /html/body/main/div[2]/section/article//h2 | /html/body/main/div[2]/section/article//h3')

            conteudo_filtrado = []  # Lista para armazenar o conteúdo filtrado
            skip_next = False  # Flag para pular o próximo índice

            for Y, x in enumerate(conteudo):    
                text = x.text.strip()
            
                if skip_next:  # Se a flag estiver ativada, pula o índice
                    skip_next = False
                    continue

                # Ignorar conteúdo com base nas condições
                if text == "":  # Verifica se o conteúdo é vazio
                    print(f"Ignorando índice {Y} devido a conteúdo vazio")
                    continue

                if "JÁ VOTARAM" in text:
                    print(f"Ignorando índice {Y} devido a 'JÁ VOTARAM'")
                    continue 

                if "VEJA TAMBÉM" in text:
                    print(f"Ignorando índice {Y} e o próximo devido a 'VEJA TAMBÉM'")
                    skip_next = True  # Define a flag para pular o próximo índice
                    continue  
                if "LEIA TAMBÉM" in text:
                    print(f"Ignorando índice {Y} e o próximo devido a 'LEIA TAMBÉM'")
                    skip_next = True  # Define a flag para pular o próximo índice
                    break  
                
                if "dizem os Fanáticos" in text:
                    print(f"Ignorando índice {Y} devido a 'dizem os Fanáticos'")
                    continue  

                if "Siga o canal do Somos" in text:
                    print(f"Ignorando índice {Y} devido a 'Siga o canal do Somos'")
                    continue  
            
                conteudo_filtrado.append(text)  # Adiciona o conteúdo

            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_filtrado
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Buscando no próximo site...")

In [ ]:
# Define o número máximo de tentativas
# BolaVip
max_tentativas = 2

# Número de tentativas já realizadas
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        driver.get("https://br.bolavip.com/noticias-recentes")
        link = driver.find_element('xpath', f'//*[@id="__next"]/main/div[2]/div/div/article[1]/figure/a').get_attribute('href')
        
        ap = driver.find_element('xpath', '/html/body/div[1]/main/div[2]/div/div/article[1]/div/a/span').get_attribute('innerText')
        ap = ap.upper()
        x = 2
        while ap == "APOSTAS" or ap == "NFL":
            link = driver.find_element('xpath', f'//*[@id="__next"]/main/div[2]/div/div/article[{x}]/figure/a').get_attribute('href')
            ap = driver.find_element('xpath', f'/html/body/div[1]/main/div[2]/div/div/article[{x}]/div/a/span').get_attribute('innerText')
            ap = ap.upper()
            x += 1
        driver.get(link)
        titulo = driver.find_element('xpath', f'/html/body/main/div[2]/section[1]/div/article/div[1]/div[2]/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', f'/html/body/main/div[2]/section[1]/div/article/div[1]/div[2]/h2').get_attribute('innerText')
            autor = driver.find_element('xpath', f'/html/body/main/div[2]/section[1]/div/article/div[1]/div[2]/div/div[1]/div/p[1]/a').get_attribute('innerText')
            try:
                img = driver.find_element("xpath", f'/html/body/main/div[2]/section[1]/div/article/div[2]/figure/img').get_attribute('srcset')
                descrimg = driver.find_element("xpath", f'/html/body/main/div[2]/section[1]/div/article/div[2]/figure/figcaption').get_attribute('innerText')
            except:
                img = "Sem imagem"
                descrimg = "Sem imagem"
            conteudo = driver.find_elements('xpath', f'/html/body/main/div[2]/section[1]/div/article//p | /html/body/main/div[2]/section[1]/div/article//h2 | /html/body/main/div[2]/section[1]/div/article//h3')

            conteudo_filtrado = []  # Lista para armazenar o conteúdo filtrado
            skip_next = False  # Flag para pular o próximo índice

            for Y, x in enumerate(conteudo):    
                text = x.text.strip()
            
                if skip_next:  # Se a flag estiver ativada, pula o índice
                    skip_next = False
                    continue

                # Ignorar conteúdo com base nas condições
                if text == "":  # Verifica se o conteúdo é vazio
                    print(f"Ignorando índice {Y} devido a conteúdo vazio")
                    continue

                if "JÁ VOTARAM" in text:
                    print(f"Ignorando índice {Y} devido a 'JÁ VOTARAM'")
                    continue 

                if "VEJA TAMBÉM" in text:
                    print(f"Ignorando índice {Y} e o próximo devido a 'VEJA TAMBÉM'")
                    skip_next = True  # Define a flag para pular o próximo índice
                    continue  
                if "LEIA TAMBÉM" in text:
                    print(f"Ignorando índice {Y} e o próximo devido a 'LEIA TAMBÉM'")
                    skip_next = True  # Define a flag para pular o próximo índice
                    break  
                
                if "dizem os Fanáticos" in text:
                    print(f"Ignorando índice {Y} devido a 'dizem os Fanáticos'")
                    continue  

                if "Siga o canal do Somos" in text:
                    print(f"Ignorando índice {Y} devido a 'Siga o canal do Somos'")
                    continue  

                if "Google News" in text:
                    print(f"Ignorando índice {Y} devido a 'Siga o canal do Somos'")
                    continue  
                if "É um jornalista que" in text:
                    print(f"Ignorando índice {Y} devido a 'Siga o canal do Somos'")
                    continue  
                if "É jornalista há" in text:
                    print(f"Ignorando índice {Y} devido a 'Siga o canal do Somos'")
                    continue  
            
                conteudo_filtrado.append(text)  # Adiciona o conteúdo

            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_filtrado
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Buscando no próximo site...")

In [ ]:
# Define o número máximo de tentativas
# Uol esportes
max_tentativas = 2

# Número de tentativas já realizadas
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        driver.get("https://www.uol.com.br/esporte/futebol/ultimas/")
        link = driver.find_element('xpath', f'/html/body/div[4]/section/section/div/div/div[1]/section/div/div/div/div/div[1]/div/a').get_attribute('href')
        driver.get(link)
        titulo = driver.find_element('xpath', f'/html/body/div[1]/main/article/div[1]/div[1]/div/div[1]/div/div[2]/div/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            titulo = driver.find_element('xpath', '/html/body/div[1]/main/article/div[1]/div[1]/div/div[1]/div/div[2]/div/h1').get_attribute('innerText')
            subtitulo = "Sem subtitulo"
            autor = driver.find_element('xpath', '/html/body/div[1]/main/article/div[1]/div[1]/div/div[2]/div/div[1]/div').get_attribute('innerText')
            try:
                img = driver.find_element('xpath', '/html/body/div[1]/main/article/div[1]/div[2]/div/div/figure/figure/picture/img').get_attribute('src')
                descrimg = driver.find_element('xpath', '/html/body/div[1]/main/article/div[1]/div[2]/div/div/figure/figure/picture/img').get_attribute('alt')
            except:
                img = "Sem imagem"
                descrimg = "Sem descrição"
                
            conteudo1 = driver.find_elements('xpath', '/html/body/div[1]/main/article/div[1]/div[2]/div/div/div//p | /html/body/div[1]/main/article/div[1]/div[2]/div/div/div//h2')
            conteudo2 = driver.find_elements('xpath', '/html/body/div[1]/main/article/div[2]/div/div/div/div[1]//p | /html/body/div[1]/main/article/div[2]/div/div/div/div[1]//h2')
            
            # Extraindo o texto dos elementos
            conteudo1_texto = [element.text for element in conteudo1]
            conteudo2_texto = [element.text for element in conteudo2]
            
            conteudo = conteudo1_texto + conteudo2_texto

            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")
        
        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Encerrando o programa.")


In [ ]:
# Define o número máximo de tentativas
# GE Globo Esporte
# Não está funcionando.
max_tentativas = 2
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        driver.get('https://ge.globo.com/')
        
        time.sleep(3)  # Aguardar o carregamento da página inicial

        # Encontrar o link da notícia e acessar
        for i in range(2, 4):
            try:
                descr = driver.find_element('xpath', f'/html/body/div[2]/main[3]/div[4]/div[2]/div/div/div/div/div/div/div/div[2]/div[1]/div/div/div[{i}]/div/div/div/div[4]/span[2]').get_attribute('innerText')
                print(descr)
            except:
                continue
            if descr == "Tempo Real":
                print("Notícia em tempo real, passando para a próxima.")
                continue
            else:
                try:
                    print(i)
                    linkg = driver.find_element('xpath', f'/html/body/div[2]/main[3]/div[4]/div[2]/div/div/div/div/div/div/div/div[2]/div[1]/div/div/div[{i}]/div/div/div/div[2]/div/h2/a').get_attribute('href')
                except:
                    print(i)
                    linkg = driver.find_element('xpath', f'/html/body/div[2]/main[3]/div[4]/div[2]/div/div/div/div/div/div/div/div[2]/div[1]/div/div/div[{i}]/div/div/div/div[3]/div/h2/a').get_attribute('href')
            
        time.sleep(1)  # Aguardar o carregamento da página da notícia
        driver.get(linkg)
        print(linkg)
        # Coletar o título da notícia
        titulo = driver.find_element('xpath', f'/html/body/div[2]/main/div[3]/div[1]/h1').get_attribute('innerText')
        print('Pegou o título')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            print('Conferiu no json')
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', '/html/body/div[2]/main/div[3]/div[2]/h2').get_attribute('innerText')
            print('Pegou sub')
            autor = driver.find_element('xpath', '/html/body/div[2]/main/div[4]/div[1]/div/div').get_attribute('innerText')
            print('Pegou autor')
            
            for l in range(1, 20):
                try:
                    img = driver.find_element('xpath', f'/html/body/div[2]/main/div[5]/article/div[4]/div[{l}]/div/div/div/figure/amp-img/img').get_attribute('src')
                    print('pegou imagem')
                    descrimg = driver.find_element('xpath', '/html/body/div[2]/main/div[5]/article/div[4]/div[{l}]/div/div/div/figure/amp-img/img').get_attribute('alt')
                except:
                    img = "Sem imagem"
                    descrimg = "Sem descrição"
                
            conteudo1 = driver.find_elements('xpath', '/html/body/div[2]/main/div[5]/article//p | /html/body/div[2]/main/div[5]/article//h2 | /html/body/div[2]/main/div[5]/article//ul | /html/body/div[2]/main/div[5]/article//blockquote | /html/body/div[2]/main/div[5]/article//span')
            print('Pegou o conteúdo')
            
            # Extraindo o texto dos elementos e filtrando conteúdo vazio
            conteudo_texto = [element.text for element in conteudo1 if element.text.strip()]
            print('Extraiu o conteúdo dos elementos e filtrou')
            
            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_texto
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")
        
        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Encerrando o programa.")


In [5]:
# Define o número máximo de tentativas
# Globo Internacional
max_tentativas = 2
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        driver.get('https://ge.globo.com/futebol/futebol-internacional/')
        time.sleep(1)  # Aguardar o carregamento da página inicial

        try:
            link = driver.find_element('xpath', f'//*[@id="feed-placeholder"]/div/div/div[2]/div/div/div/div[1]/div/div/div/div[3]/div/h2/a').get_attribute('href')
        except:
            try:
                link = driver.find_element('xpath', f'/html/body/div[2]/main[3]/div[4]/div[2]/div/div/div/div/div/div/div/div[2]/div/div/div/div[1]/div/div/div/div[2]/div/h2/a').get_attribute('href')
            except:
                link = driver.find_element('xpath', f'/html/body/div[2]/main[3]/div[4]/div[2]/div/div/div/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/h2/a').get_attribute('href')

        # Acessar o link da notícia
        driver.get(link)
        time.sleep(1)  # Aguardar o carregamento da página da notícia

        # Coletar o título da notícia
        titulo = driver.find_element('xpath', f'/html/body/div[2]/main/div[3]/div[1]/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', '/html/body/div[2]/main/div[3]/div[2]/h2').get_attribute('innerText')
            autor = driver.find_element('xpath', '/html/body/div[2]/main/div[4]/div[1]/div/div').get_attribute('innerText')
            
            try:
                img = driver.find_element('xpath', '/html/body/div[2]/main/div[5]/article//img').get_attribute('src')
                descrimg = driver.find_element('xpath', '/html/body/div[2]/main/div[5]/article//img').get_attribute('alt')
            except:
                img = "Sem imagem"
                descrimg = "Sem descrição"
                
            conteudo1 = driver.find_elements('xpath', '/html/body/div[2]/main/div[5]/article//p | /html/body/div[2]/main/div[5]/article//h2 | /html/body/div[2]/main/div[5]/article//ul | /html/body/div[2]/main/div[5]/article//blockquote | /html/body/div[2]/main/div[5]/article//span')
            
            # Extraindo o texto dos elementos e filtrando conteúdo vazio
            conteudo_texto = [element.text for element in conteudo1 if element.text.strip()]
            
            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_texto
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Encerrando o programa.")


Tentativa 1 de 2

Notícia adicionada ao JSON - Quem manda no futebol da Europa? Confira raio-X de clubes, empresas, investidores e "SAFs"
BUSCANDO PRÓXIMO SITE... 



In [7]:
# Define o número máximo de tentativas
max_tentativas = 2
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        driver.get('https://ge.globo.com/futebol/selecao-brasileira/')
        time.sleep(1)  # Aguardar o carregamento da página inicial

        # Acessar o link da notícia principal
        try:
            link = driver.find_element('xpath', '/html/body/div[2]/main[3]/div[4]/div[2]/div/div/div/div/div/div/div/div[2]/div/div/div/div[1]/div/div/div/div[2]/div/h2/a').get_attribute('href')
        except Exception as e:
            print(f"Erro ao tentar encontrar o link da notícia: {e}")
            print("Tentando acessar um link alternativo...")
            continue  # Pula para a próxima tentativa

        # Acessar o link da notícia
        driver.get(link)
        time.sleep(1)  # Aguardar o carregamento da página da notícia

        # Coletar o título da notícia
        titulo = driver.find_element('xpath', '/html/body/div[2]/main/div[3]/div[1]/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', '/html/body/div[2]/main/div[3]/div[2]/h2').get_attribute('innerText')
            autor = driver.find_element('xpath', '/html/body/div[2]/main/div[4]/div[1]/div/div').get_attribute('innerText')
            
            try:
                img = driver.find_element('xpath', '/html/body/div[2]/main/div[5]/article//img').get_attribute('src')
                descrimg = driver.find_element('xpath', '/html/body/div[2]/main/div[5]/article//img').get_attribute('alt')
            except:
                img = "Sem imagem"
                descrimg = "Sem descrição"
                
            conteudo1 = driver.find_elements('xpath', '/html/body/div[2]/main/div[5]/article//p | /html/body/div[2]/main/div[5]/article//h2 | /html/body/div[2]/main/div[5]/article//ul | /html/body/div[2]/main/div[5]/article//blockquote | /html/body/div[2]/main/div[5]/article//span')
            
            # Extraindo o texto dos elementos e filtrando conteúdo vazio
            conteudo_texto = [element.text for element in conteudo1 if element.text.strip()]
            
            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_texto
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Encerrando o programa.")

Tentativa 1 de 2

Notícia adicionada ao JSON - Dorival Júnior faz balanço positivo do ano da Seleção: "Muitas coisas mudaram"
BUSCANDO PRÓXIMO SITE... 



In [9]:
# Define o número máximo de tentativas
max_tentativas = 2
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        # Acessa a página principal de esportes do IG
        driver.get("https://esporte.ig.com.br/futebol/")
        time.sleep(2)  # Aguardar o carregamento da página inicial

        # Localiza o link da notícia principal e acessa
        try:
            link = driver.find_element('xpath', '/html/body/main/div/div[1]/div/div[3]/div/div/div/div/ul[1]/li[1]/a').get_attribute('href')
        except Exception as e:
            print(f"Erro ao tentar encontrar o link da notícia: {e}")
            print("Tentando acessar um link alternativo...")
            link = driver.find_element('xpath', '/html/body/main/div/div[1]/div/div[2]/div/div/div/div/ul[1]/div[2]/a').get_attribute('href')
            continue  # Pula para a próxima tentativa

        driver.get(link)
        time.sleep(1)  # Aguardar o carregamento da página da notícia

        # Coleta o título da notícia
        titulo = driver.find_element('xpath', '/html/body/div[3]/div/div[1]/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', '/html/body/div[3]/div/div[1]/h2').get_attribute('innerText')
            autor = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div/div').get_attribute('innerText')
            
            try:
                # Coleta imagem e descrição da imagem, se disponíveis
                img = driver.find_element('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div/div[1]/figure/div/img').get_attribute('src')
                descrimg = driver.find_element('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div/div[1]/figure/div/img').get_attribute('alt')
            except:
                img = "Sem imagem"
                descrimg = "Sem descrição"
            
            # Coleta o conteúdo da notícia
            conteudo1 = driver.find_elements('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div//p | /html/body/main/div/div/div[1]/div/section/div[2]/div//h2 | /html/body/main/div/div/div[1]/div/section/div[2]/div//ul | /html/body/main/div/div/div[1]/div/section/div[2]/div//blockquote | /html/body/main/div/div/div[1]/div/section/div[2]/div//span')

            # Filtra o texto dos elementos, removendo conteúdos vazios
            conteudo_texto = [element.text for element in conteudo1 if element.text.strip()]
            
            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_texto
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Encerrando o programa.")

Tentativa 1 de 2

Notícia adicionada ao JSON - Thiago Silva faz forte desabafo após empate, e Mano Menezes pede cautela em relação aos jovens
BUSCANDO PRÓXIMO SITE... 



In [11]:
# Define o número máximo de tentativas
max_tentativas = 2
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        # Acessa a página principal de esportes do IG
        driver.get("https://esporte.ig.com.br/futebol/internacional/")
        time.sleep(2)  # Aguardar o carregamento da página inicial

        # Localiza o link da notícia principal e acessa
        try:
            link = driver.find_element('xpath', '/html/body/main/div/div[1]/div/div/div[2]/div/div/div/ul[1]/li[1]/a').get_attribute('href')
        except Exception as e:
            print(f"Erro ao tentar encontrar o link da notícia: {e}")
            print("Tentando acessar um link alternativo...")
            link = driver.find_element('xpath', '/html/body/main/div/div[1]/div/div[2]/div/div/div/div/ul[1]/div[2]/a').get_attribute('href')
            continue  # Pula para a próxima tentativa

        driver.get(link)
        time.sleep(1)  # Aguardar o carregamento da página da notícia

        # Coleta o título da notícia
        titulo = driver.find_element('xpath', '/html/body/div[3]/div/div[1]/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', '/html/body/div[3]/div/div[1]/h2').get_attribute('innerText')
            autor = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div/div').get_attribute('innerText')
            
            try:
                # Coleta imagem e descrição da imagem, se disponíveis
                img = driver.find_element('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div/div[1]/figure/div/img').get_attribute('src')
                descrimg = driver.find_element('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div/div[1]/figure/div/img').get_attribute('alt')
            except:
                img = "Sem imagem"
                descrimg = "Sem descrição"
            
            # Coleta o conteúdo da notícia
            conteudo1 = driver.find_elements('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div//p | /html/body/main/div/div/div[1]/div/section/div[2]/div//h2 | /html/body/main/div/div/div[1]/div/section/div[2]/div//ul | /html/body/main/div/div/div[1]/div/section/div[2]/div//blockquote | /html/body/main/div/div/div[1]/div/section/div[2]/div//span')

            # Filtra o texto dos elementos, removendo conteúdos vazios
            conteudo_texto = [element.text for element in conteudo1 if element.text.strip()]
            
            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_texto
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Encerrando o programa.")

Tentativa 1 de 2

Notícia adicionada ao JSON - Neymar gera festa em treino do Al-Hilal com gol; veja vídeo
BUSCANDO PRÓXIMO SITE... 



In [13]:
# Define o número máximo de tentativas
max_tentativas = 2
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        # Acessa a página da seleção do IG
        driver.get("https://esporte.ig.com.br/futebol/selecaobrasileira/")
        time.sleep(2)  # Aguardar o carregamento da página inicial

        # Localiza o link da notícia principal e acessa
        try:
            link = driver.find_element('xpath', '/html/body/main/div/div[1]/div/div/div[2]/div/div/div/ul[1]/li[1]/a').get_attribute('href')
        except Exception as e:
            print(f"Erro ao tentar encontrar o link da notícia: {e}")
            print("Tentando acessar um link alternativo...")
            link = driver.find_element('xpath', '/html/body/main/div/div[1]/div/div[2]/div/div/div/div/ul[1]/div[2]/a').get_attribute('href')
            continue  # Pula para a próxima tentativa

        driver.get(link)
        time.sleep(1)  # Aguardar o carregamento da página da notícia

        # Coleta o título da notícia
        titulo = driver.find_element('xpath', '/html/body/div[3]/div/div[1]/h1').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            subtitulo = driver.find_element('xpath', '/html/body/div[3]/div/div[1]/h2').get_attribute('innerText')
            autor = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div/div').get_attribute('innerText')
            
            try:
                # Coleta imagem e descrição da imagem, se disponíveis
                img = driver.find_element('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div/div[1]/figure/div/img').get_attribute('src')
                descrimg = driver.find_element('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div/div[1]/figure/div/img').get_attribute('alt')
            except:
                img = "Sem imagem"
                descrimg = "Sem descrição"
            
            # Coleta o conteúdo da notícia
            conteudo1 = driver.find_elements('xpath', '/html/body/main/div/div/div[1]/div/section/div[2]/div//p | /html/body/main/div/div/div[1]/div/section/div[2]/div//h2 | /html/body/main/div/div/div[1]/div/section/div[2]/div//ul | /html/body/main/div/div/div[1]/div/section/div[2]/div//blockquote | /html/body/main/div/div/div[1]/div/section/div[2]/div//span')

            # Filtra o texto dos elementos, removendo conteúdos vazios
            conteudo_texto = [element.text for element in conteudo1 if element.text.strip()]
            
            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "subtitulo": subtitulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_texto
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Encerrando o programa.")

Tentativa 1 de 2

Notícia adicionada ao JSON - Brasil vence a Bolívia no sufoco e ganha fôlego no Sul-Americano Sub-20
BUSCANDO PRÓXIMO SITE... 



In [26]:
# Define o número máximo de tentativas
max_tentativas = 2
tentativa = 0

while tentativa < max_tentativas:
    try:
        tentativa += 1
        print(f"Tentativa {tentativa} de {max_tentativas}")

        # Extrai os títulos já presentes no arquivo para verificar notícias duplicadas
        titulos_existentes = {noticia['titulo'] for noticia in noticias}

        # Acessa a página do onefootball
        driver.get("https://onefootball.com/pt-br/inicio")
        time.sleep(2)  # Aguardar o carregamento da página inicial

        # Localiza o link da notícia principal e acessa
        try:
            d = driver.find_element('xpath', '/html/body/div[1]/main/div/div/div[4]/div/section/ul/li[2]/article/a[2]/p[1]/span[1]')
            link = driver.find_element('xpath', '/html/body/div[1]/main/div/div/div[4]/div/section/ul/li[3]/article/a[2]').get_attribute('href')
        except Exception as e:
            try:
                link = driver.find_element('xpath', '/html/body/div[1]/main/div/div/div[4]/div/section/ul/li[2]/article/a[1]').get_attribute('href')
            except:
                print(f"Erro ao tentar encontrar o link da notícia: {e}")
                print("Tentando acessar um link alternativo...")
                link = driver.find_element('xpath', '/html/body/div[1]/main/div/div/div[4]/div/section/ul/li[2]/article/a[2]').get_attribute('href')
                continue  # Pula para a próxima tentativa

        driver.get(link)
        time.sleep(1)  # Aguardar o carregamento da página da notícia

        # Coleta o título da notícia
        titulo = driver.find_element('xpath', '/html/body/div[1]/main/div/div[1]/div[3]/div/article/div[1]/div[2]/h2').get_attribute('innerText')

        # Verifica se o título já está no arquivo JSON
        if titulo not in titulos_existentes:
            # Se a notícia não estiver no arquivo, coleta o restante das informações
            autor = driver.find_element('xpath', '/html/body/div[1]/main/div/div[1]/div[3]/div/article/div[1]/div[2]/div[1]/div').get_attribute('innerText')
            
            try:
                # Coleta imagem e descrição da imagem, se disponíveis
                img = driver.find_element('xpath', '/html/body/div[1]/main/div/div[1]/div[3]/div/article/div[2]/div/picture/img').get_attribute('src')
                descrimg = driver.find_element('xpath', '/html/body/div[1]/main/div/div[1]/div[3]/div/article/div[2]/div/picture/img').get_attribute('alt')
            except:
                img = "Sem imagem"
                descrimg = "Sem descrição"
            
            # Coleta o conteúdo da notícia
            conteudo1 = driver.find_elements(
                'xpath',
                '''
                /html/body/div[1]/main/div/div[1]/div[4]/div[1]/div
                //p[not(ancestor::section)] |
                /html/body/div[1]/main/div/div[1]/div[4]/div[1]/div
                //h2[not(ancestor::section)] |
                /html/body/div[1]/main/div/div[1]/div[4]/div[1]/div
                //ul[not(ancestor::section)] |
                /html/body/div[1]/main/div/div[1]/div[4]/div[1]/div
                //blockquote[not(ancestor::section)] |
                /html/body/div[1]/main/div/div[1]/div[4]/div[1]/div
                //span[not(ancestor::section)]
                '''
            )


            # Filtra o texto dos elementos, removendo conteúdos vazios
            conteudo_texto = [element.text for element in conteudo1 if element.text.strip()]
            
            # Cria um dicionário com as informações da notícia
            noticia = {
                "titulo": titulo,
                "autor": autor,
                "img": img,
                "descrimg": descrimg,
                "conteudo": conteudo_texto
            }

            # Adiciona a nova notícia ao array de notícias
            noticias.append(noticia)

            # Salva o arquivo JSON com a nova notícia imediatamente
            with open(json_file, 'w', encoding='utf-8') as file:
                json.dump(noticias, file, ensure_ascii=False, indent=4)

            print(f"\nNotícia adicionada ao JSON - {titulo}")
            print("==========================================================================================")
            
            print("BUSCANDO PRÓXIMO SITE... \n")
        else:
            print(f"Notícia '{titulo}' já existe no arquivo. Ignorando...")
            print("BUSCANDO PRÓXIMO SITE... \n")

        # Se o código for executado sem exceções, sai do loop
        break

    except Exception as e:
        print(f"Erro encontrado: {e}")
        print("Tentando novamente...\n")
        time.sleep(3)  # Espera 3 segundos antes de tentar novamente

if tentativa == max_tentativas:
    print("Número máximo de tentativas atingido. Encerrando o programa.")

Tentativa 1 de 2

Notícia adicionada ao JSON - Brasil vence a Bolívia no sufoco e ganha fôlego no Sul-Americano Sub-20
BUSCANDO PRÓXIMO SITE... 

